In [10]:
import zipfile

local_zip = "rps.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("tmp/rps-train")
zip_ref.close()

local_zip = "rps-test-set.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("tmp/rps-test")
zip_ref.close()

In [11]:
import os

base_dir = "tmp/rps-train/rps"

rock_dir = os.path.join(base_dir, "rock")
paper_dir = os.path.join(base_dir, "paper")
scissors_dir = os.path.join(base_dir, "scissors")

In [12]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(3, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 74, 74, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 17, 17, 128)      

In [13]:
model.compile(loss ="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "tmp/rps-train/rps"
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

valid_dir = "tmp/rps-test/rps-test-set"
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    class_mode="categorical",
    batch_size=128
)

valid_gen = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150,150),
    class_mode="categorical",
    batch_size=128
)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [15]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("accuracy") >= .95:
            self.model.stop_training = True

callback = myCallback()

history = model.fit(train_gen,
                   epochs=20,
                   steps_per_epoch=20,
                   validation_data = valid_gen,
                   verbose=1,
                   validation_steps=3,
                   callbacks=callback)

Epoch 1/20
20/20 [==============================] - 82s 4s/step - loss: 1.1874 - accuracy: 0.3341 - val_loss: 1.0916 - val_accuracy: 0.5511
Epoch 2/20
20/20 [==============================] - 69s 3s/step - loss: 1.1027 - accuracy: 0.3603 - val_loss: 1.0931 - val_accuracy: 0.3656
Epoch 3/20
20/20 [==============================] - 65s 3s/step - loss: 1.0600 - accuracy: 0.4520 - val_loss: 1.0474 - val_accuracy: 0.3978
Epoch 4/20
20/20 [==============================] - 65s 3s/step - loss: 1.0641 - accuracy: 0.4452 - val_loss: 1.1305 - val_accuracy: 0.3333
Epoch 5/20
20/20 [==============================] - 64s 3s/step - loss: 0.9591 - accuracy: 0.5238 - val_loss: 0.8431 - val_accuracy: 0.6989
Epoch 6/20
20/20 [==============================] - 64s 3s/step - loss: 0.7885 - accuracy: 0.6397 - val_loss: 0.7127 - val_accuracy: 0.6694
Epoch 7/20
20/20 [==============================] - 64s 3s/step - loss: 0.7639 - accuracy: 0.6794 - val_loss: 0.6797 - val_accuracy: 0.7070
Epoch 8/20
20/20 [==

In [16]:
zipfile.ZipFile("./rps-validation.zip", "r").extractall("./rps-validation")

In [32]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

val_pics = os.listdir("./rps-validation")

for pic in val_pics:
    path = os.path.join("./rps-validation", pic)
    img = load_img(path, target_size=(150, 150))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    

    
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(pic)
    print(classes)

1/1 [==============================] - 0s 22ms/step
paper-hires1.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
paper-hires2.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
paper1.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
paper2.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
paper3.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
paper4.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
paper5.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 23ms/step
paper6.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
paper7.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 22ms/step
paper8.png
[[1. 0. 0.]]
1/1 [==============================] - 0s 21ms/step
paper9.png
[[0. 1. 0.]]
1/1 [==============================] - 0s 22ms/step
rock-hires1.png
[[0. 1. 0.]]
1/1 [==============================] - 0s 24ms/step
rock-hires2.png
[[0

In [62]:
import cv2
from IPython.display import clear_output
import time 

cap = cv2.VideoCapture(0)
cap.set(3,640) # adjust width
cap.set(4,480) # adjust height

def useModel(img):
    img = tf.image.resize(img, (150, 150))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    c = model.predict(images)
    print(c)
    
try:
    while True:
        success, img = cap.read()
        cv2.imshow("Webcam", img) # This will open an independent window
        useModel(img);
        if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
            cap.release()
            break
except Exception as e:
    cv2.destroyWindow("Webcam") 
    cv2.waitKey(1)
    print(e)
        
cv2.destroyWindow("Webcam") 
cv2.waitKey(1)

1/1 [==============================] - 0s 48ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 42ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[1.0000000e+00 2.8668235e-13 0.0000000e+00]]
1/1 [==============================] - 0s 27ms/step
[[1.0000000e+00 2.8175696e-35 0.0000000e+00]]
1/1 [==============================] - 0s 27ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 28ms/step
[[1.0000000e+00 1.1987024e-31 0.0000000e+00]]
1/1 [==============================] - 0s 29ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 33ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 37ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 38ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 29ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 29ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 36ms/step
[[1.

1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 39ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 36ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 45ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 36ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 37ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 41ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [====================

1/1 [==============================] - 0s 36ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 38ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 39ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1. 0.]]
1/1 [====================

[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [=======

1/1 [==============================] - 0s 34ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 29ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 29ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 28ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 28ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 31ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [====================

...

1/1 [==============================] - 0s 37ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 34ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 34ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 36ms/step
[[1.000000e+00 5.126596e-14 0.000000e+00]]
1/1 [==============================] - 0s 34ms/step
[[1.0000000e+00 2.0391578e-28 0.0000000e+00]]
1/1 [==============================] - 0s 32ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 33ms/step
[[1.0000000e+00 1.1881879e-20 0.0000000e+00]]
1/1 [==============================] - 0s 34ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 32ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 37ms/step
[[9.999943e-01 5.684993e-06 0.000000e+00]]
1/1 [==============================] - 0s 37ms/step
[[1.0000000e+00 1.7041402e-25 0.0000000e+00]]
1/1 [==============================] - 0s 33ms/step
[[7.0705872e-31 1.0000000e+00 0.0000000e+00]]
1/1 [=======================

1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [====================

1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 36ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 40ms/step
[[0. 1. 0.]]
1/1 [====================

1/1 [==============================] - 0s 37ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 36ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 32ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 36ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 34ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 33ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1. 0.]]
1/1 [====================

-1

In [60]:
# import cv2
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# cap = cv2.VideoCapture(0)
# cap.set(3,640) # adjust width
# cap.set(4,480) # adjust height

# def useModel(img):
#     path = "temp.jpg"
#     cv2.imwrite(path, img)
#     img = load_img(path, target_size=(150, 150))
#     x = img_to_array(img)
#     x = np.expand_dims(x, axis=0)
    
#     images = np.vstack([x])
#     c = model.predict(images, batch_size=10)
#     os.remove("temp.jpg")
#     print(c)
    
    
# try:
#     while True:
#         success, img = cap.read()
#         cv2.imshow("Webcam", img) # This will open an independent window
#         useModel(img);
#         if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
#             cap.release()
#             break
# except Exception as e:
#     cv2.destroyAllWindows() 
#     cv2.waitKey(1)
#     print(e)
        
# cv2.destroyAllWindows() 
# cv2.waitKey(1)

1/1 [==============================] - 0s 36ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 35ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 25ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 24ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 24ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 24ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 25ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 31ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 25ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [====================

1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 38ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [====================

1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 29ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [====================

1/1 [==============================] - 0s 30ms/step
[[9.9999595e-01 4.0222076e-06 0.0000000e+00]]
1/1 [==============================] - 0s 31ms/step
[[9.9998534e-01 1.4608002e-05 0.0000000e+00]]
1/1 [==============================] - 0s 30ms/step
[[2.7493436e-26 1.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 36ms/step
[[2.3548215e-29 1.0000000e+00 0.0000000e+00]]
1/1 [==============================] - 0s 35ms/step
[[1.0000000e+00 1.9960571e-22 0.0000000e+00]]
1/1 [==============================] - 0s 31ms/step
[[9.639305e-17 1.000000e+00 0.000000e+00]]
1/1 [==============================] - 0s 31ms/step
[[1.0000000e+00 1.8902081e-25 0.0000000e+00]]
1/1 [==============================] - 0s 30ms/step
[[1.0000000e+00 4.0618634e-30 0.0000000e+00]]
1/1 [==============================] - 0s 30ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 32ms/step
[[1. 0. 0.]]
1/1 [==============================] - 0s 31ms/step
[[1. 0. 0.]]
1/1 [===================

1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [====================

1/1 [==============================] - 0s 28ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 30ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]
1/1 [==============================] - 0s 27ms/step
[[0. 1. 0.]]


-1